In [1]:
import os
import sys
BASE_DIR = r"C:\Users\nino.zhou\Desktop\my_project\cp_newbank_mobile_behavior"
sys.path.insert(0, BASE_DIR)

import pandas as pd
import numpy as np
from config import setting
from config import setting
import pandas as pd
import itertools
from sklearn.externals import joblib
import datetime
# All Includes

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

import os

C:\Users\nino.zhou\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


## 使用论文数据集：

In [11]:
# All Includes

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

import os

In [12]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

# Output classes to learn how to classify
LABELS = [
    "WALKING", 
    "WALKING_UPSTAIRS", 
    "WALKING_DOWNSTAIRS", 
    "SITTING", 
    "STANDING", 
    "LAYING"
] 

In [13]:
DATASET_PATH = setting.BASE_DIR + "/data_model/data/UCI HAR Dataset/"
print("\n" + "Dataset is now located at: " + DATASET_PATH)


Dataset is now located at: C:\Users\nino.zhou\Desktop\my_project\cp_newbank_mobile_behavior/data_model/data/UCI HAR Dataset/


In [14]:
TRAIN = "train/"
TEST = "test/"


# Load "X" (the neural network's training and testing inputs)

def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        file = open(signal_type_path, 'r')
        # Read dataset from disk, dealing with text files' syntax
        X_signals.append(
            [np.array(serie, dtype=np.float32) for serie in [
                row.replace('  ', ' ').strip().split(' ') for row in file
            ]]
        )
        file.close()
    
    return np.transpose(np.array(X_signals), (1, 2, 0))

X_train_signals_paths = [
    DATASET_PATH + TRAIN + "Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + TEST + "Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

X_train = load_X(X_train_signals_paths)
X_test = load_X(X_test_signals_paths)


# Load "y" (the neural network's training and testing outputs)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]], 
        dtype=np.int32
    )
    file.close()
    
    # Substract 1 to each output class for friendly 0-based indexing 
    return y_ - 1

y_train_path = DATASET_PATH + TRAIN + "y_train.txt"
y_test_path = DATASET_PATH + TEST + "y_test.txt"

y_train = load_y(y_train_path)
y_test = load_y(y_test_path)

# 构造和新网银行数据集一样的样本
# 新网银行数据只有6个列，顺序分别是acc_x, acc_y, acc_z，acc_gx, acc_gy, acc_gz，因此这里要切割+顺序调整
X_test = X_test[:, :, [6, 7, 8, 3, 4, 5]]
X_train = X_train[:, :, [6, 7, 8, 3, 4, 5]]  
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7352, 128, 6), (2947, 128, 6), (7352, 1), (2947, 1))

## Additionnal Parameters:

Here are some core parameter definitions for the training. 

For example, the whole neural network's structure could be summarised by enumerating those parameters and the fact that two LSTM are used one on top of another (stacked) output-to-input as hidden layers through time steps. 

In [15]:
# Input Data 

training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # 128 timesteps per series
n_input = len(X_train[0][0])  # 9 input parameters per timestep


# LSTM Neural Network's internal structure

n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 128
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")


Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(2947, 128, 6) (2947, 1) 0.14890043 0.46668795
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


## Utility functions for training:

In [2]:
class BiLstmResNet(object):
    def __init__(self, _X, n_classes):
        # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters. 
        # Moreover, two LSTM cells are stacked which adds deepness to the neural network. 
        # Note, some code of this notebook is inspired from an slightly different 
        # RNN architecture used on another dataset, some of the credits goes to 
        # "aymericdamien" under the MIT license.

        # (NOTE: This step could be greatly optimised by shaping the dataset once
        # ReLU activation, thanks to Yu Zhao for adding this improvement here:
        weights = tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
        biases = tf.Variable(tf.random_normal([n_hidden]))
        _X = tf.nn.relu(tf.matmul(_X, weights) + biases)

        # Split data because rnn cell needs a list of inputs for the RNN inner loop
        # new shape: n_steps * (batch_size, n_hidden)


        # bilstm + resnet残差网络
        for i in range(2):
            with tf.name_scope("bi_resnet_" + str(i)):
                X_shortcut = _X
                with tf.name_scope("bi_lstm1") as scope:
                    with tf.name_scope("lstm_fw"):
                        lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
                    with tf.name_scope("lstm_bw"):
                        lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
                    outputs, _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, _X, dtype = tf.float32, scope=scope)
                    outputs = tf.add(outputs[0], outputs[1])

                with tf.name_scope("bi_lstm2") as scope:
                    with tf.name_scope("lstm_fw"):
                        lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
                    with tf.name_scope("lstm_bw"):
                        lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
                    outputs, _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, outputs, dtype = tf.float32, scope=scope)
                    outputs = tf.concat(outputs, 2)

                with tf.name_scope("resnet"):
                    outputs = tf.reshape(outputs, [-1, 2 * n_hidden])
                    weights = tf.Variable(tf.random_normal([2 * n_hidden, n_hidden]))
                    biases = tf.Variable(tf.random_normal([n_hidden]))
                    outputs = tf.nn.relu(tf.matmul(outputs, weights) + biases)
                    outputs = tf.reshape(outputs, [-1, n_steps, n_hidden]) 
                    _X = tf.add(X_shortcut, outputs)

                with tf.name_scope("BN") as scope:
                    _X = tf.nn.relu(tf.layers.batch_normalization(_X, training = True))
                    _X = tf.nn.dropout(_X, 0.7)

        with tf.name_scope("bi_resnet_last"):
            X_shortcut = _X
            with tf.name_scope("bi_lstm1") as scope:
                with tf.name_scope("lstm_fw"):
                    lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
                with tf.name_scope("lstm_bw"):
                    lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
                outputs, _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, _X, dtype = tf.float32, scope=scope)
                outputs = tf.add(outputs[0], outputs[1])

            with tf.name_scope("bi_lstm2") as scope:
                with tf.name_scope("lstm_fw"):
                    lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
                with tf.name_scope("lstm_bw"):
                    lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
                outputs, _ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, outputs, dtype = tf.float32, scope=scope)

            # 最后一层只取正向的lstm输出，且不走resnet
            _X = outputs[0]

        # Get last time step's output feature for a "many-to-one" style classifier, 
        # as in the image describing RNNs at the top of this page
        # Linear activation
        lstm_last_output = _X[:, -1, :]
        with tf.name_scope("pretrain_pooler"):
            weights = tf.Variable(tf.random_normal([n_hidden, 6], mean=1.0))
            biases = tf.Variable(tf.random_normal([6]))
            self.pretrain_pooled_output = tf.matmul(lstm_last_output, weights) + biases
            
        with tf.name_scope("finturn_pooler"):
            weights = tf.Variable(tf.random_normal([n_hidden, 19], mean=1.0))
            biases = tf.Variable(tf.random_normal([19]))
            self.finturn_pooled_output = tf.matmul(lstm_last_output, weights) + biases
            
    def get_pretrain_pooled_output(self):
        return self.pretrain_pooled_output
    
    def get_finturn_pooled_output(self):
        return self.finturn_pooled_output


def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data. 
    
    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index] 

    return batch_s


def one_hot(y_, n_classes):
    # Function to encode neural one-hot output labels from number indexes 
    # e.g.: 
    # one_hot(y_=[[5], [0], [3]], n_classes=6):
    #     return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    
    y_ = y_.reshape(len(y_))
    return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  # Returns FLOATS


## Let's get serious and build the neural network:

In [17]:
# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

model = BiLstmResNet(x, n_classes)
pred = model.get_pretrain_pooled_output()

# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

pred_class = tf.argmax(pred,1)

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.cont

## Hooray, now train the neural network:

In [18]:
# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

# Perform Training steps with "batch_size" amount of example data at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs = extract_batch_size(X_train, step, batch_size)
    batch_ys = one_hot(extract_batch_size(y_train, step, batch_size), n_classes)

    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs, 
            y: batch_ys
        }
    )

    train_losses.append(loss)
    train_accuracies.append(acc)
    
    # Evaluate network only at some steps for faster training: 
    if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):
        
        # To not spam console, show training accuracy/loss in this "if"
        print("Training iter #" + str(step*batch_size) + \
              ":   Batch Loss = " + "{:.6f}".format(loss) + \
              ", Accuracy = {}".format(acc))
        
        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy], 
            feed_dict={
                x: X_test,
                y: one_hot(y_test, n_classes)
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("PERFORMANCE ON TEST SET: " + \
              "Batch Loss = {}".format(loss) + \
              ", Accuracy = {}".format(acc))

    step += 1

print("Optimization Finished!")

# Accuracy for test data

one_hot_predictions, pred_class, accuracy, final_loss = sess.run(
    [pred, pred_class, accuracy, cost],
    feed_dict={
        x: X_test,
        y: one_hot(y_test, n_classes),
    }
)

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("FINAL RESULT: " + \
      "Batch Loss = {}".format(final_loss) + \
      ", Accuracy = {}".format(accuracy))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce MX130, pci bus id: 0000:02:00.0, compute capability: 5.0

Training iter #128:   Batch Loss = 7.508059, Accuracy = 0.125
PERFORMANCE ON TEST SET: Batch Loss = 7.139112949371338, Accuracy = 0.41499829292297363
Training iter #240000:   Batch Loss = 1.090782, Accuracy = 0.8359375
PERFORMANCE ON TEST SET: Batch Loss = 1.4388999938964844, Accuracy = 0.798439085483551
Training iter #480000:   Batch Loss = 0.348855, Accuracy = 1.0
PERFORMANCE ON TEST SET: Batch Loss = 0.7315038442611694, Accuracy = 0.866644024848938
Training iter #720000:   Batch Loss = 0.212389, Accuracy = 1.0
PERFORMANCE ON TEST SET: Batch Loss = 0.6937472820281982, Accuracy = 0.8724126219749451
Training iter #960000:   Batch Loss = 0.192828, Accuracy = 1.0
PERFORMANCE ON TEST SET: Batch Loss = 0.6115515232086182, Accuracy = 0.8785205483436584
Training iter #1200000:   Batch Loss = 0.454187, Accuracy = 0.875
PERFORMANCE ON TEST SET: Batc

## 模型保存

In [19]:
saver = tf.train.Saver()  # 创建保存对象
saver.save(sess, setting.BASE_DIR + "/data_model/model/bilstm_resnet_model/model.ckpt")

'C:\\Users\\nino.zhou\\Desktop\\my_project\\cp_newbank_mobile_behavior/data_model/model/bilstm_resnet_model/model.ckpt'

## Training is good, but having visual insight is even better:

Okay, let's plot this simply in the notebook for now.

In [ ]:
# (Inline plots: )
%matplotlib inline

font = {
    'family' : 'Bitstream Vera Sans',
    'weight' : 'bold',
    'size'   : 18
}
matplotlib.rc('font', **font)

width = 12
height = 12
plt.figure(figsize=(width, height))

indep_train_axis = np.array(range(batch_size, (len(train_losses)+1)*batch_size, batch_size))
plt.plot(indep_train_axis, np.array(train_losses),     "b--", label="Train losses")
plt.plot(indep_train_axis, np.array(train_accuracies), "g--", label="Train accuracies")

indep_test_axis = np.append(
    np.array(range(batch_size, len(test_losses)*display_iter, display_iter)[:-1]),
    [training_iters]
)
plt.plot(indep_test_axis, np.array(test_losses),     "b-", label="Test losses")
plt.plot(indep_test_axis, np.array(test_accuracies), "g-", label="Test accuracies")

plt.title("Training session's progress over iterations")
plt.legend(loc='upper right', shadow=True)
plt.ylabel('Training Progress (Loss or Accuracy values)')
plt.xlabel('Training iteration')

plt.show()

## And finally, the multi-class confusion matrix and metrics!

In [ ]:
# Results

predictions = one_hot_predictions.argmax(1)

print("Testing Accuracy: {}%".format(100*accuracy))

print("")
print("Precision: {}%".format(100*metrics.precision_score(y_test, predictions, average="weighted")))
print("Recall: {}%".format(100*metrics.recall_score(y_test, predictions, average="weighted")))
print("f1_score: {}%".format(100*metrics.f1_score(y_test, predictions, average="weighted")))

print("")
print("Confusion Matrix:")
confusion_matrix = metrics.confusion_matrix(y_test, predictions)
print(confusion_matrix)
normalised_confusion_matrix = np.array(confusion_matrix, dtype=np.float32)/np.sum(confusion_matrix)*100

print("")
print("Confusion matrix (normalised to % of total test data):")
print(normalised_confusion_matrix)
print("Note: training and testing data is not equally distributed amongst classes, ")
print("so it is normal that more than a 6th of the data is correctly classifier in the last category.")

# Plot Results: 
width = 12
height = 12
plt.figure(figsize=(width, height))
plt.imshow(
    normalised_confusion_matrix, 
    interpolation='nearest', 
    cmap=plt.cm.rainbow
)
plt.title("Confusion matrix \n(normalised to % of total test data)")
plt.colorbar()
tick_marks = np.arange(n_classes)
LABELS = range(19)
plt.xticks(tick_marks, LABELS, rotation=90)
plt.yticks(tick_marks, LABELS)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.grid()
plt.show()

In [ ]:
import pandas as pd
pd.DataFrame(confusion_matrix)

In [ ]:
sess.close()

## Conclusion

Outstandingly, **the final accuracy is of 91%**! And it can peak to values such as 93.25%, at some moments of luck during the training, depending on how the neural network's weights got initialized at the start of the training, randomly. 

This means that the neural networks is almost always able to correctly identify the movement type! Remember, the phone is attached on the waist and each series to classify has just a 128 sample window of two internal sensors (a.k.a. 2.56 seconds at 50 FPS), so it amazes me how those predictions are extremely accurate given this small window of context and raw data. I've validated and re-validated that there is no important bug, and the community used and tried this code a lot. (Note: be sure to report something in the issue tab if you find bugs, otherwise [Quora](https://www.quora.com/), [StackOverflow](https://stackoverflow.com/questions/tagged/tensorflow?sort=votes&pageSize=50), and other [StackExchange](https://stackexchange.com/sites#science) sites are the places for asking questions.)

I specially did not expect such good results for guessing between the labels "SITTING" and "STANDING". Those are seemingly almost the same thing from the point of view of a device placed at waist level according to how the dataset was originally gathered. Thought, it is still possible to see a little cluster on the matrix between those classes, which drifts away just a bit from the identity. This is great.

It is also possible to see that there was a slight difficulty in doing the difference between "WALKING", "WALKING_UPSTAIRS" and "WALKING_DOWNSTAIRS". Obviously, those activities are quite similar in terms of movements. 

I also tried my code without the gyroscope, using only the 3D accelerometer's 6 features (and not changing the training hyperparameters), and got an accuracy of 87%. In general, gyroscopes consumes more power than accelerometers, so it is preferable to turn them off. 


## Improvements

In [another open-source repository of mine](https://github.com/guillaume-chevalier/HAR-stacked-residual-bidir-LSTMs), the accuracy is pushed up to nearly 94% using a special deep LSTM architecture which combines the concepts of bidirectional RNNs, residual connections, and stacked cells. This architecture is also tested on another similar activity dataset. It resembles the nice architecture used in "[Google’s Neural Machine Translation System: Bridging the Gap between Human and Machine Translation](https://arxiv.org/pdf/1609.08144.pdf)", without an attention mechanism, and with just the encoder part - as a "many to one" architecture instead of a "many to many" to be adapted to the Human Activity Recognition (HAR) problem. I also worked more on the problem and came up with the [LARNN](https://github.com/guillaume-chevalier/Linear-Attention-Recurrent-Neural-Network), however it's complicated for just a little gain. Thus the current, original activity recognition project is simply better to use for its outstanding simplicity. 

If you want to learn more about deep learning, I have also built a list of the learning ressources for deep learning which have revealed to be the most useful to me [here](https://github.com/guillaume-chevalier/Awesome-Deep-Learning-Resources). 


## References

The [dataset](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) can be found on the UCI Machine Learning Repository: 

> Davide Anguita, Alessandro Ghio, Luca Oneto, Xavier Parra and Jorge L. Reyes-Ortiz. A Public Domain Dataset for Human Activity Recognition Using Smartphones. 21th European Symposium on Artificial Neural Networks, Computational Intelligence and Machine Learning, ESANN 2013. Bruges, Belgium 24-26 April 2013.

The RNN image for "many-to-one" is taken from Karpathy's post: 

> Andrej Karpathy, The Unreasonable Effectiveness of Recurrent Neural Networks, 2015, 
> http://karpathy.github.io/2015/05/21/rnn-effectiveness/

## Citation

Copyright (c) 2016 Guillaume Chevalier. To cite my code, you can point to the URL of the GitHub repository, for example: 

> Guillaume Chevalier, LSTMs for Human Activity Recognition, 2016, 
> https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition

My code is available for free and even for private usage for anyone under the [MIT License](https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition/blob/master/LICENSE), however I ask to cite for using the code. 

Here is the BibTeX citation code: 
```
@misc{chevalier2016lstms,
  title={LSTMs for human activity recognition},
  author={Chevalier, Guillaume},
  year={2016}
}
```

## Extra links

### Connect with me

- [LinkedIn](https://ca.linkedin.com/in/chevalierg)
- [Twitter](https://twitter.com/guillaume_che)
- [GitHub](https://github.com/guillaume-chevalier/)
- [Quora](https://www.quora.com/profile/Guillaume-Chevalier-2)
- [YouTube](https://www.youtube.com/c/GuillaumeChevalier)
- [Dev/Consulting](http://www.neuraxio.com/en/)

### Liked this project? Did it help you? Leave a [star](https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition/stargazers), [fork](https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition/network/members) and share the love!

This activity recognition project has been seen in:

- [Hacker News 1st page](https://news.ycombinator.com/item?id=13049143)
- [Awesome TensorFlow](https://github.com/jtoy/awesome-tensorflow#tutorials)
- [TensorFlow World](https://github.com/astorfi/TensorFlow-World#some-useful-tutorials)
- And more.

---


In [ ]:
# Let's convert this notebook to a README automatically for the GitHub project's title page:
!jupyter nbconvert --to markdown LSTM.ipynb
!mv LSTM.md README.md

# 精调模型

## 加载新网银行比赛数据集

In [3]:
arr_feature = np.load(os.path.join(setting.BASE_DIR, "./data_model/data/lstm_arr_x.npy"))
arr_y_true = np.load(os.path.join(setting.BASE_DIR, "./data_model/data/lstm_arr_y.npy"))
arr_fragment_ids = np.load(os.path.join(setting.BASE_DIR, "./data_model/data/lstm_arr_fragment_ids.npy"))
arr_feature = arr_feature.reshape(-1, 61, 7)
arr_feature = arr_feature[:, :, 1:]

X_train = arr_feature[:5000]
X_test = arr_feature[5000:]
y_train = arr_y_true[:5000]
y_test = arr_y_true[5000:]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5000, 61, 6), (2292, 61, 6), (5000,), (2292,))

## 参数配置

In [4]:
# Input Data 
training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # 128 timesteps per series
n_input = len(X_train[0][0])  # 9 input parameters per timestep

# LSTM Neural Network's internal structure

n_hidden = 32 # Hidden layer num of features
n_classes = 19 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 128
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(2292, 61, 6) (2292,) -0.0012841467812171117 0.983350085738804
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


## 模型构造

In [5]:
# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

model = BiLstmResNet(x, n_classes)
pred = model.get_finturn_pooled_output()  # 输出19类别维度的结果

# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

pred_class = tf.argmax(pred,1)

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.cont

## 加载预训练模型

In [6]:
# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce MX130, pci bus id: 0000:02:00.0, compute capability: 5.0



In [7]:
arr = tf.get_default_graph().get_tensor_by_name("Variable:0").eval()
pd.DataFrame(arr)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.698951,-0.433187,-0.728653,0.637444,-1.668677,0.255249,0.659610,-0.847741,-1.806465,0.187687,...,-1.057081,0.969517,0.235178,-0.007478,0.419809,-0.586455,-0.942344,0.796461,0.417467,-1.158699
1,-1.326119,1.812539,-0.905138,-0.449598,-0.121010,-0.158090,-0.864567,0.391304,-0.463028,-0.687384,...,0.463098,2.138987,-2.515596,0.206002,-0.118785,-0.698603,0.945583,1.502327,0.746327,-0.627463
2,-1.079807,-0.924794,-0.922195,0.464371,1.627714,1.197855,-0.193896,-0.736550,0.798999,0.017732,...,-0.595296,0.058631,0.249398,-1.923150,-0.603278,-0.949498,0.712092,0.504264,-0.400131,0.332546
3,-0.166143,1.464615,-1.427495,-0.290825,-0.100847,-0.159492,1.452873,0.332313,3.103173,0.210181,...,-1.028448,-0.317208,0.870311,1.355028,-1.166253,1.723183,-0.036836,-0.147719,-0.209817,-0.020769
4,1.293911,0.104622,0.716802,0.824855,0.414387,-0.771556,1.275928,-0.275397,-1.078982,-0.719323,...,0.062931,0.157228,-1.192283,-0.826715,-0.130886,0.286609,-0.210561,-0.528799,0.288088,-0.010277
5,-0.408117,-0.336523,0.816087,1.292082,-0.471701,-0.183618,-0.325382,1.314073,-0.849194,-1.133268,...,0.416888,1.516812,0.405853,0.667030,-0.810618,2.647501,-1.371370,-0.784657,0.773238,-1.222035


In [9]:
arr = tf.get_default_graph().get_tensor_by_name("Variable:0").eval()
pd.DataFrame(arr)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.006285,0.294867,-0.085339,4.754151e-26,0.030528,-0.003832,0.102384,-0.120494,0.015287,-0.183406,...,-0.068499,-0.030219,0.121295,0.649131,-3.835617e-34,0.001921,1.494776,-1.047291e-05,0.012167,-0.860775
1,-0.090696,0.287738,-0.046503,-2.049580e-26,0.524261,-0.449429,-0.555833,0.026392,-0.350846,0.301399,...,-0.134333,-0.549066,0.104371,0.093799,2.075366e-33,-0.154621,0.599389,4.072649e-10,0.071788,0.375207
2,0.292271,0.090162,-0.055480,-4.949238e-27,-0.313873,-0.428764,0.399681,-0.448918,-0.222726,0.289480,...,-0.198499,-0.244478,0.208726,0.049530,2.541380e-34,-0.011812,0.811600,1.853219e-10,0.131350,-0.711814
3,0.007997,0.237111,0.172313,9.283945e-25,-0.006330,0.023712,0.301037,-0.083233,0.048220,0.532171,...,-0.062085,0.240714,0.017378,-0.032796,-1.288876e-33,-0.334259,0.164963,3.389346e-05,-0.034422,0.193228
4,0.036175,-0.387973,-0.134112,-7.632570e-27,0.120555,0.035381,0.025918,0.195004,0.134977,-0.235438,...,-0.049537,-0.072281,-0.001138,0.012646,5.177031e-35,-0.270583,-0.336023,7.768295e-10,0.013277,-0.259227
5,0.028055,0.353466,-0.227519,1.753321e-27,-0.153987,0.023347,0.051298,-0.226944,-0.125159,-0.141367,...,-0.468389,0.255448,0.012762,-0.217748,-4.307898e-34,0.181763,-0.178315,1.286192e-09,0.025379,0.188053


In [11]:
import re
import collections
tvars = tf.trainable_variables()
name_to_variable = collections.OrderedDict()
for var in tvars:
    name = var.name
    m = re.match("^(.*):\\d+$", name)
    if m is not None:
        name = m.group(1)
    name_to_variable[name] = var

init_checkpoint = setting.BASE_DIR + "/data_model/model/bilstm_resnet_model/model.ckpt"
init_vars = tf.train.list_variables(init_checkpoint)
import collections
assignment_map = collections.OrderedDict()
for x in init_vars:
    (name, var) = (x[0], x[1])
    if name in name_to_variable:
        assignment_map[name] = name_to_variable[name]
        
tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
sess.run(tf.global_variables_initializer())  # 再次运行global_variables_initializer()，使加载的模型参数生效

OrderedDict([('Variable',
              <tf.Variable 'Variable:0' shape=(6, 32) dtype=float32_ref>),
             ('Variable_1',
              <tf.Variable 'Variable_1:0' shape=(32,) dtype=float32_ref>),
             ('batch_normalization/beta',
              <tf.Variable 'batch_normalization/beta:0' shape=(32,) dtype=float32_ref>),
             ('batch_normalization/gamma',
              <tf.Variable 'batch_normalization/gamma:0' shape=(32,) dtype=float32_ref>),
             ('batch_normalization_1/beta',
              <tf.Variable 'batch_normalization_1/beta:0' shape=(32,) dtype=float32_ref>),
             ('batch_normalization_1/gamma',
              <tf.Variable 'batch_normalization_1/gamma:0' shape=(32,) dtype=float32_ref>),
             ('bi_resnet_0/bi_lstm1//bw/basic_lstm_cell/bias',
              <tf.Variable 'bi_resnet_0/bi_lstm1//bw/basic_lstm_cell/bias:0' shape=(128,) dtype=float32_ref>),
             ('bi_resnet_0/bi_lstm1//bw/basic_lstm_cell/kernel',
              <tf.Var

In [8]:
saver = tf.train.Saver(tf.global_variables())
moudke_file=tf.train.latest_checkpoint(setting.BASE_DIR + "/data_model/model/bilstm_resnet_model/")
saver.restore(sess, moudke_file)

INFO:tensorflow:Restoring parameters from C:\Users\nino.zhou\Desktop\my_project\cp_newbank_mobile_behavior/data_model/model/bilstm_resnet_model/model.ckpt


In [10]:
# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

# Perform Training steps with "batch_size" amount of example data at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs = extract_batch_size(X_train, step, batch_size)
    batch_ys = one_hot(extract_batch_size(y_train, step, batch_size), n_classes)

    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs, 
            y: batch_ys
        }
    )

    train_losses.append(loss)
    train_accuracies.append(acc)
    
    # Evaluate network only at some steps for faster training: 
    if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):
        
        # To not spam console, show training accuracy/loss in this "if"
        print("Training iter #" + str(step*batch_size) + \
              ":   Batch Loss = " + "{:.6f}".format(loss) + \
              ", Accuracy = {}".format(acc))
        
        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy], 
            feed_dict={
                x: X_test,
                y: one_hot(y_test, n_classes)
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("PERFORMANCE ON TEST SET: " + \
              "Batch Loss = {}".format(loss) + \
              ", Accuracy = {}".format(acc))

    step += 1

print("Optimization Finished!")

# Accuracy for test data

one_hot_predictions, pred_class, accuracy, final_loss = sess.run(
    [pred, pred_class, accuracy, cost],
    feed_dict={
        x: X_test,
        y: one_hot(y_test, n_classes),
    }
)

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("FINAL RESULT: " + \
      "Batch Loss = {}".format(final_loss) + \
      ", Accuracy = {}".format(accuracy))

Training iter #128:   Batch Loss = 3.046315, Accuracy = 0.046875
PERFORMANCE ON TEST SET: Batch Loss = 3.007279396057129, Accuracy = 0.16972076892852783
Training iter #240000:   Batch Loss = 1.745227, Accuracy = 0.4375
PERFORMANCE ON TEST SET: Batch Loss = 1.6268775463104248, Accuracy = 0.49389180541038513
Training iter #480000:   Batch Loss = 1.656731, Accuracy = 0.4453125
PERFORMANCE ON TEST SET: Batch Loss = 1.5663750171661377, Accuracy = 0.5244328379631042
Training iter #720000:   Batch Loss = 1.534860, Accuracy = 0.59375
PERFORMANCE ON TEST SET: Batch Loss = 1.5320751667022705, Accuracy = 0.5641361474990845
Training iter #960000:   Batch Loss = 1.442707, Accuracy = 0.5390625
PERFORMANCE ON TEST SET: Batch Loss = 1.5075749158859253, Accuracy = 0.5763525366783142
Training iter #1200000:   Batch Loss = 1.312786, Accuracy = 0.625
PERFORMANCE ON TEST SET: Batch Loss = 1.449526309967041, Accuracy = 0.6051483154296875
Training iter #1440000:   Batch Loss = 1.376604, Accuracy = 0.609375
P

In [11]:
# 保存预测类别结果
df_train = pd.read_csv(setting.BASE_DIR + "\data_model\data\sensor_train.csv")
arr_test_fragment_id = arr_fragment_ids[5000:]

df_train["category_bilstm_resnet"] = None
for fragment_id, category in zip(arr_test_fragment_id, pred_class):
    index = df_train[df_train["fragment_id"] == fragment_id].index
    df_train.loc[index, "category_bilstm_resnet"] = category
df_train

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,category_bilstm_resnet
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0,None
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0,None
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0,None
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0,None
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0,None
...,...,...,...,...,...,...,...,...,...,...
425354,7291,4561,-0.1,-0.5,-1.7,-0.2,3.4,7.4,18,None
425355,7291,4647,0.7,-1.9,1.1,0.5,1.7,10.7,18,None
425356,7291,4735,1.3,-1.4,-0.5,1.4,2.1,8.0,18,None
425357,7291,4830,1.2,-1.6,-0.1,1.6,1.8,9.2,18,None


In [15]:
# 构造行号（因为元数据就是按顺序排列，因此不需重置顺序）
df_train = df_train.reset_index()
df_train = df_train.rename(columns={"index": "id"})

df_train = df_train[["id", "fragment_id", "time_point", "behavior_id", "category_bilstm_resnet"]]
df_train.to_csv(setting.BASE_DIR + "\data_model\data\category_bilstm_resnet.csv")